In [1]:
%display latex
from delierium import *
from IPython.core.debugger import set_trace

In [2]:
x1,x2,x3=var("x1 x2 x3")
f       = function("f")(x1,x2,x3)
M=[x1*x1*x2*x2*x3**3, x1**3 * x3**3, x1**3 * x2*x3, x2*x3]

In [3]:
M[0].show()

x1^2*x2^2*x3^3

In [4]:
def complete(m, context = None):
    '''
    Given a set {m1,m2,...} of monomials, the complete set corresponding to m is returned
    >>> x1,x2,x3=var("x1 x2 x3")
    >>> f       = function("f")(x1,x2,x3)
    >>> M=[x1*x1*x2*x2*x3**3, x1**3 * x3**3, x1**3 * x2*x3, x2*x3]
    [ x1*x2^2*x3^3,x1^3*x3^3,x1^2*x2*x3^2,x2^2*x3^3, x1^2*x2*x3^3, x1*x2*x3, x1^2*x2*x3,
    x1^3*x2*x3,x2*x3^2,x1*x2*x3^2,x1^2*x2^2*x3^3,x1^3*x2*x3^2,x2*x3^3, x1*x2*x3^3, x1^3*x2*x3^3, x2*x3]
    '''
    from functools import cmp_to_key
    # S1
    vars = []
    m = set(m)
    for _ in m:
        vars.extend(list(_.variables()))
    vars = list(set(vars))
    _bool = lambda a, b: bool (a < b)
    vars = sorted (vars, key = cmp_to_key (_bool), reverse = True)
    while 1:
        res = [(_, multipliers(_, m, vars)) for _ in m]
        m0 = set()
        # S2
        for (monomial, (_multipliers, _nonmultipliers)) in res:
            m0 |= set([monomial * n for n in _nonmultipliers])
        to_remove = set()
        for _m0  in m0:
            # S3
            # check whether in class of any of the monomials
            for (monomial, (_multipliers, _nonmultipliers)) in res:
                if all(_m0.degree(x) >= monomial.degree(x) for x in _multipliers) and \
                   all(_m0.degree(x) == monomial.degree(x) for x in _nonmultipliers):
                    to_remove.add(_m0)
        m0 -= to_remove
        if not m0:
            return sorted(list(m))
        m |= m0

In [5]:
c=complete (M)

In [6]:
for _c in sorted(c):
    print(_c)

x1*x2*x3^3
x1^3*x3^3
x1^3*x2*x3^2
x1*x2^2*x3^3
x1^3*x2*x3
x1^2*x2*x3
x1*x2*x3
x2*x3^3
x1^3*x2*x3^3
x1^2*x2*x3^3
x2*x3^2
x2^2*x3^3
x2*x3
x1^2*x2*x3^2
x1*x2*x3^2
x1^2*x2^2*x3^3


In [7]:
import operator
def hausnummer (list_of_derivatives, context = Context((f,), (x1,x2,x3))):
    monomials = []
    for d in list_of_derivatives:
        der = derivative_to_vec(d, context)
        mon = reduce (operator.__mul__ , [a**b for a, b in zip (context._independent, der)], 1)
        monomials.append (mon)
    c = complete (monomials)
    print (c)
    for mausi in c:
        gurki = [mausi.degree (_) for _ in context._independent]
    return gurki

In [8]:
hausnummer([diff(f,x1,x3,x3), diff(f, x3, x2, x2, x1)])

[x1*x3^2, x1*x2^2*x3]


[1, 2, 1]

In [9]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
# ctx: items are in descending order ?
ctx = Context((w,z), (x,y), Mgrlex)
g1=diff(z,y,y)+diff(z,y)/(2*y)
g2=diff(w,x,x)+(4*y**2)*diff(w,y)-(8*y**2)*diff(z,x)-8*y*w
g3=diff(w,x,y)-diff(z,x,x)/2-diff(w,x)/(2*y)-diff(z,y)*6*y**2
g4=diff(w,y,y)-2*diff(z,x,y)-diff(w,y)/(2*y)-w/(2*y**2)
l = [Differential_Polynomial(_, ctx) for _ in [g1,g2,g3,g4]]


In [27]:
def derivative_to_monomial(d, context):
     return reduce (operator.__mul__ , [a**b for a, b in zip (context._independent, derivative_to_vec (d, context))], 1)
        
derivative_to_monomial (diff(z, x,x,x,x,y,y), ctx).show()
        
def complete1(m, context = None):
    '''
    Given a set {m1,m2,...} of monomials, the complete set corresponding to m is returned
    >>> x1,x2,x3=var("x1 x2 x3")
    >>> f       = function("f")(x1,x2,x3)
    >>> M=[x1*x1*x2*x2*x3**3, x1**3 * x3**3, x1**3 * x2*x3, x2*x3]
    [ x1*x2^2*x3^3,x1^3*x3^3,x1^2*x2*x3^2,x2^2*x3^3, x1^2*x2*x3^3, x1*x2*x3, x1^2*x2*x3,
    x1^3*x2*x3,x2*x3^2,x1*x2*x3^2,x1^2*x2^2*x3^3,x1^3*x2*x3^2,x2*x3^3, x1*x2*x3^3, x1^3*x2*x3^3, x2*x3]
    '''
    from functools import cmp_to_key
    # S1
    vars = []
    for s in m:
        s.show()
        
    m = [derivative_to_monomial (_.Lder(), context) for _ in m]
    
    print (m)    
    vars = list(set(vars))
    _bool = lambda a, b: bool (a < b)
    vars = sorted (context._independent, key = cmp_to_key (_bool), reverse = True)
    print ("vars:", vars)    
    while 1:
        #set_trace()
        res = [(_, multipliers(_, m, vars)) for _ in m]
        m0 = set()
        # S2
        for (monomial, (_multipliers, _nonmultipliers)) in res:
            print (derivative_to_vec (monomial, context))
            print (monomial,  reduce (operator.__mul__ , [a**b for a, b in zip (context._independent, derivative_to_vec (monomial, context))], 1)
                  )
            m0 |= set([monomial * n for n in _nonmultipliers])
        to_remove = set()
        for _m0  in m0:
            # S3
            # check whether in class of any of the monomials
            for (monomial, (_multipliers, _nonmultipliers)) in res:
                if all(_m0.degree(x) >= monomial.degree(x) for x in _multipliers) and \
                   all(_m0.degree(x) == monomial.degree(x) for x in _nonmultipliers):
                    to_remove.add(_m0)
        m0 -= to_remove
        if not m0:
            return sorted(list(m))
        m |= m0

x^4*y^2

In [17]:
for _ in l:
    _.show()

1/2*diff(z(x, y), y)/y + diff(z(x, y), y, y)

4*y^2*diff(w(x, y), y) - 8*y^2*diff(z(x, y), x) - 8*y*w(x, y) + diff(w(x, y), x, x)

-6*y^2*diff(z(x, y), y) - 1/2*diff(w(x, y), x)/y + diff(w(x, y), x, y) - 1/2*diff(z(x, y), x, x)

-1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), y, y) - 2*diff(z(x, y), x, y)

In [28]:
def algorithm27 (S, context):
    s = {}
    for _ in S:
        _fun = _.Lfunc()
        s.setdefault(_fun, []).append(_)
    res = []
    for k in s:
        if len(s[k]) == 1:
            res.extend (s[k])
            continue
        print ("=====================>", k)
        _ = complete1(s[k], context)
        res.extend(_)
    return Reorder(res, context, ascending=True)
algorithm27(l, ctx)

=====================> w


4*y^2*diff(w(x, y), y) - 8*y^2*diff(z(x, y), x) - 8*y*w(x, y) + diff(w(x, y), x, x)

-6*y^2*diff(z(x, y), y) - 1/2*diff(w(x, y), x)/y + diff(w(x, y), x, y) - 1/2*diff(z(x, y), x, x)

-1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), y, y) - 2*diff(z(x, y), x, y)

[x^2, x*y, y^2]
vars: [x, y]
[0]
x^2 1
[0, 0]
x*y 1
[0]
y^2 1


AttributeError: 'sage.symbolic.expression.Expression' object has no attribute '_p'

In [ ]:
derivative